In [1]:
import polars as pl

from spells import summon, ColName, ColType, ColSpec

### Metagame Summary

In this notebook I'll demonstrate the metagame summaries I like to look at, along the lines of the "Deck Color" page on 17Lands.com. First, lets generate that table. I like to only look at the top player cohort to judge what is effective for a winning strategy. Here are recipes for the basic charts

In [2]:
filter_spec = {ColName.PLAYER_COHORT: "Top"}
columns = [ColName.NUM_WON, ColName.NUM_GAMES, ColName.GAME_WR]

print(
    summon(
        "DSK", columns=columns, group_by=[ColName.NUM_COLORS], filter_spec=filter_spec
    )
)

two_color_filter = {"$and": [{"num_colors": 2}, filter_spec]}
print(
    summon(
        "DSK",
        columns=columns,
        group_by=[ColName.MAIN_COLORS],
        filter_spec=two_color_filter,
    )
)

shape: (5, 4)
┌────────────┬─────────┬───────────┬──────────┐
│ num_colors ┆ num_won ┆ num_games ┆ game_wr  │
│ ---        ┆ ---     ┆ ---       ┆ ---      │
│ u32        ┆ u32     ┆ u32       ┆ f64      │
╞════════════╪═════════╪═══════════╪══════════╡
│ 1          ┆ 1803    ┆ 2873      ┆ 0.627567 │
│ 2          ┆ 112978  ┆ 182958    ┆ 0.617508 │
│ 3          ┆ 12108   ┆ 20721     ┆ 0.584335 │
│ 4          ┆ 337     ┆ 600       ┆ 0.561667 │
│ 5          ┆ 17      ┆ 37        ┆ 0.459459 │
└────────────┴─────────┴───────────┴──────────┘
shape: (10, 4)
┌─────────────┬─────────┬───────────┬──────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  │
│ ---         ┆ ---     ┆ ---       ┆ ---      │
│ str         ┆ u32     ┆ u32       ┆ f64      │
╞═════════════╪═════════╪═══════════╪══════════╡
│ BG          ┆ 9587    ┆ 15624     ┆ 0.613607 │
│ BR          ┆ 11486   ┆ 18595     ┆ 0.617693 │
│ RG          ┆ 16179   ┆ 26464     ┆ 0.611359 │
│ UB          ┆ 4104    ┆ 6945      ┆ 0.590929 │
│ 

You'll note the winrates are higher than the "Top" cohort on 17Lands. That's because we are compelled to use an in-sample cohort. We might want some additional context about the archetypes, like how often the decks splash, the average number of lands, and the average converted mana cost. Let's add all three. There isn't a `GAME_SUM` column built in for splash count, so we'll define one.

In [3]:
ext = {
    "num_splash": ColSpec(col_type=ColType.GAME_SUM, expr=pl.col(ColName.HAS_SPLASH)),
    "splash_rate": ColSpec(
        col_type=ColType.AGG, expr=pl.col("num_splash") / pl.col(ColName.NUM_GAMES)
    ),
}
print(
    by_num_colors_df := summon(
        "DSK",
        columns=columns + ["splash_rate"],
        group_by=[ColName.NUM_COLORS],
        filter_spec=filter_spec,
        extensions=ext,
    )
)
print(
    by_color_pair_df := summon(
        "DSK",
        columns=columns + ["splash_rate"],
        group_by=["main_colors"],
        filter_spec=two_color_filter,
        extensions=ext,
    )
)

shape: (5, 5)
┌────────────┬─────────┬───────────┬──────────┬─────────────┐
│ num_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ splash_rate │
│ ---        ┆ ---     ┆ ---       ┆ ---      ┆ ---         │
│ u32        ┆ u32     ┆ u32       ┆ f64      ┆ f64         │
╞════════════╪═════════╪═══════════╪══════════╪═════════════╡
│ 1          ┆ 1803    ┆ 2873      ┆ 0.627567 ┆ 0.788723    │
│ 2          ┆ 112978  ┆ 182958    ┆ 0.617508 ┆ 0.306595    │
│ 3          ┆ 12108   ┆ 20721     ┆ 0.584335 ┆ 0.15757     │
│ 4          ┆ 337     ┆ 600       ┆ 0.561667 ┆ 0.526667    │
│ 5          ┆ 17      ┆ 37        ┆ 0.459459 ┆ 0.0         │
└────────────┴─────────┴───────────┴──────────┴─────────────┘
shape: (10, 5)
┌─────────────┬─────────┬───────────┬──────────┬─────────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ splash_rate │
│ ---         ┆ ---     ┆ ---       ┆ ---      ┆ ---         │
│ str         ┆ u32     ┆ u32       ┆ f64      ┆ f64         │
╞═════════════╪═════════╪═══════════╪

Columns for the average number of lands and the average mana value of spells are also included with Spells, and that might be interesting here:

In [4]:
print(
    summon(
        "BLB",
        columns=[
            ColName.NUM_WON,
            ColName.NUM_GAMES,
            ColName.GAME_WR,
            "splash_rate",
            ColName.DECK_LANDS_AVG,
            ColName.DECK_MANA_VALUE_AVG,
        ],
        group_by=[ColName.MAIN_COLORS],
        filter_spec=two_color_filter,
        extensions=ext,
    )
)

shape: (10, 7)
┌─────────────┬─────────┬───────────┬──────────┬─────────────┬────────────────┬────────────────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ splash_rate ┆ deck_lands_avg ┆ deck_mana_value_av │
│ ---         ┆ ---     ┆ ---       ┆ ---      ┆ ---         ┆ ---            ┆ g                  │
│ str         ┆ u32     ┆ u32       ┆ f64      ┆ f64         ┆ f64            ┆ ---                │
│             ┆         ┆           ┆          ┆             ┆                ┆ f64                │
╞═════════════╪═════════╪═══════════╪══════════╪═════════════╪════════════════╪════════════════════╡
│ BG          ┆ 19447   ┆ 31416     ┆ 0.619016 ┆ 0.487363    ┆ 16.775306      ┆ 2.852575           │
│ BR          ┆ 9771    ┆ 15515     ┆ 0.629778 ┆ 0.1534      ┆ 16.714663      ┆ 2.732122           │
│ RG          ┆ 6235    ┆ 10421     ┆ 0.598311 ┆ 0.486038    ┆ 16.811534      ┆ 2.829357           │
│ UB          ┆ 7610    ┆ 12480     ┆ 0.609776 ┆ 0.332212    ┆ 16.828686    

The metagame share is also important. Polars provides for in-column aggregations that are broadcast back to the row level, which will allow us to do this easily.

In [5]:
ext = {
    "num_games_total": ColSpec(col_type=ColType.AGG, expr=pl.col("num_games").sum()),
    "metagame_share": ColSpec(
        col_type=ColType.AGG, expr=pl.col("num_games") / pl.col("num_games_total")
    ),
}

print(
    summon(
        "DSK",
        columns=[ColName.NUM_WON, ColName.NUM_GAMES, ColName.GAME_WR, "metagame_share"],
        group_by=[ColName.MAIN_COLORS],
        filter_spec=two_color_filter,
        extensions=ext,
    )
)

shape: (10, 5)
┌─────────────┬─────────┬───────────┬──────────┬────────────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ metagame_share │
│ ---         ┆ ---     ┆ ---       ┆ ---      ┆ ---            │
│ str         ┆ u32     ┆ u32       ┆ f64      ┆ f64            │
╞═════════════╪═════════╪═══════════╪══════════╪════════════════╡
│ BG          ┆ 9587    ┆ 15624     ┆ 0.613607 ┆ 0.085397       │
│ BR          ┆ 11486   ┆ 18595     ┆ 0.617693 ┆ 0.101635       │
│ RG          ┆ 16179   ┆ 26464     ┆ 0.611359 ┆ 0.144645       │
│ UB          ┆ 4104    ┆ 6945      ┆ 0.590929 ┆ 0.03796        │
│ UG          ┆ 16478   ┆ 26363     ┆ 0.625043 ┆ 0.144093       │
│ UR          ┆ 6821    ┆ 11537     ┆ 0.591228 ┆ 0.063058       │
│ WB          ┆ 5632    ┆ 9460      ┆ 0.595349 ┆ 0.051706       │
│ WG          ┆ 4287    ┆ 7219      ┆ 0.59385  ┆ 0.039457       │
│ WR          ┆ 18239   ┆ 28903     ┆ 0.631042 ┆ 0.157976       │
│ WU          ┆ 20165   ┆ 31848     ┆ 0.633164 ┆ 0.174073    

Let's look at the wr and metagame evolution for one color pair over the format. This is the subject of a nice two-dimensional line chart that you'll have to figure out how to make.

In [6]:
print(
    summon(
        "DSK",
        columns=[ColName.GAME_WR, "metagame_share"],
        group_by=[ColName.MAIN_COLORS, ColName.FORMAT_WEEK],
        filter_spec={"$and": [two_color_filter, {"main_colors": "WU"}]},
        extensions=ext,
    )
)

shape: (5, 4)
┌─────────────┬─────────────┬──────────┬────────────────┐
│ main_colors ┆ format_week ┆ game_wr  ┆ metagame_share │
│ ---         ┆ ---         ┆ ---      ┆ ---            │
│ str         ┆ i64         ┆ f64      ┆ f64            │
╞═════════════╪═════════════╪══════════╪════════════════╡
│ WU          ┆ 1           ┆ 0.644531 ┆ 0.305451       │
│ WU          ┆ 2           ┆ 0.631017 ┆ 0.247205       │
│ WU          ┆ 3           ┆ 0.631833 ┆ 0.195303       │
│ WU          ┆ 4           ┆ 0.624798 ┆ 0.136241       │
│ WU          ┆ 5           ┆ 0.619848 ┆ 0.1158         │
└─────────────┴─────────────┴──────────┴────────────────┘
